# Regression

In [4]:
import learning
from IPython.core import display as ICD
import pandas as pd
from mord import OrdinalRidge
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor
pd.options.display.max_columns = 999
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from tqdm import tqdm
from sklearn.externals import joblib
import copy

models = [
    LinearRegression(),
    MLPRegressor() 
    MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(20, 20), random_state=1),
    KNeighborsClassifier(5)
]

transforms = [
    SelectKBest(f_regression, k=5),
    SelectKBest(f_regression, k=6)
    SelectKBest(f_regression, k=7),
    SelectKBest(f_regression, k=8),
    SelectKBest(f_regression, k=9),
    SelectKBest(f_regression, k=10),
    SelectKBest(f_regression, k=11),
    SelectKBest(f_regression, k=12),
    SelectKBest(f_regression, k=13),
    SelectKBest(f_regression, k=14),
    SelectKBest(f_regression, k=15),
    SelectKBest(f_regression, k=16)
]

def trystuff(output, train, test, models, transforms):
    minscore = 10000
    modeldict = {}
    _, Xtrain, ytrain = learning.prepare_base_data(train, output=output)
    transforms = copy.deepcopy(transforms)
    for transform in transforms:
        transform.fit(Xtrain, ytrain)
        Xt = transform.transform(Xtrain)
        models = copy.deepcopy(models)
        for model in models:
            model.fit(Xt, ytrain)
            trues, preds, score = learning.score(test, {output: model}, {output: transform})
            modeldict[(transform, model)] = score
            if(score < minscore):
                besttrans = transform
                bestmodel = model
                minscore = score
    return modeldict, bestmodel, besttrans, minscore

train = learning.get_base_data(2015)
test = learning.get_base_data(2014)

outputs = ["Goals", "Assists", "Shots", "Blocks"]
fmodels = {}
ftrans = {}
fscores = {}

print('Starting...')
for output in outputs:
    print(output)
    modeldict, m, trans, score = trystuff(output, train, test, models, transforms)
    print(m)
    print(trans)
    print(score)
    joblib.dump(modeldict, output + "_" + "modeldict.p")
    fmodels[output] = m
    ftrans[output] = trans
    fscores[output] = score
    
print(fmodels)
print(ftrans)
true, pred, score = learning.score(test, fmodels, ftrans)
print(score)

Starting...
Goals
Goals
MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
SelectKBest(k=6, score_func=<function f_regression at 0x000001B5C4E0B840>)
25.8965517241
Assists
Assists
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
SelectKBest(k=5, score_func=<function f_regression at 0x000001B5C4E0B840>)
21.6781609195
Shots
Shots
MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
    